In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
%load_ext tensorboard

In [2]:
print("Drunk Philosophers")

data_raw = pd.read_csv("./data/philosophy_data.csv")
data_raw = data_raw.drop(['sentence_spacy', 'original_publication_date', 'corpus_edition_date', 'sentence_length', 'sentence_lowered', 'lemmatized_str'], axis=1)
philosophers = list(dict.fromkeys(data_raw['author'].tolist()))

selected_data = data_raw.loc[data_raw['author'] == "Plato"]
text = ' '.join(selected_data['sentence_str'].tolist())

print(text[:2000])

Drunk Philosophers
 What's new, Socrates, to make you leave your usual haunts in the Lyceum and spend your time here by the king archon's court? Surely you are not prosecuting anyone before the king archon as I am? The Athenians do not call this a prosecution but an indictment, Euthyphro. What is this you say? Someone must have indicted you, for you are not going to tell me that you have indicted someone else. But someone else has indicted you? I do not really know him myself, Euthyphro. He is apparently young and unknown. They call him Meletus, I believe. He belongs to the Pitthean deme, if you know anyone from that deme called Meletus, with long hair, not much of a beard, and a rather aquiline nose. I don't know him, Socrates. What charge does he bring against you? A not ignoble one I think , for it is no small thing for a young man to have knowledge of such an important subject. He says he knows how our young men are corrupted and who corrupts them. He is likely to be wise, and when

In [3]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)

    diversity = 0.25
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + seqlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [ ]:
for phil in philosophers:
    print("Training: ", phil)
    selected_data = data_raw.loc[data_raw['author'] == phil]
    text = ' '.join(selected_data['sentence_str'].tolist())

    chars = sorted(list(set(text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    seqlen = 100
    dropoutFactor = 0.2
    step = seqlen
    sentences = []
    for i in range(0, len(text) - seqlen - 1, step):
        sentences.append(text[i: i + seqlen + 1])

    x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
            x[i, t, char_indices[char_in]] = 1
            y[i, t, char_indices[char_out]] = 1


    model = Sequential()
    model.add(LSTM(256, input_shape=(seqlen, len(chars)), return_sequences=True))
    model.add(Dropout(dropoutFactor))  
    model.add(LSTM(256, input_shape=(seqlen, len(chars)), return_sequences=True))
    model.add(Dropout(dropoutFactor))
    model.add(Dense(len(chars), activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.01),
        metrics=['categorical_crossentropy', 'accuracy']
    )

    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model.fit(x, y,
              batch_size=128,
              epochs=50,
              callbacks=[print_callback])

    loc = "models6/" + phil +"v6"
    print("saving to: ", loc)
    model.save(loc)

Training:  Plato


/tmp/ipykernel_1185061/2660070471.py:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
/tmp/ipykernel_1185061/2660070471.py:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
2022-04-30 05:58:27.606703: I tensorflow/cor

Epoch 1/50


2022-04-30 05:58:32.537168: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8302
2022-04-30 05:58:32.716691: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


348/348 [==============================] - ETA: 0s - loss: 2.1827 - categorical_crossentropy: 2.1827 - accuracy: 0.3768
----- Generating text after Epoch: 0
----- diversity: 0.25
----- Generating with seed: " of discipline is shameful, as I was saying earlier, and so they enslave men who are better by natur"
 of discipline is shameful, as I was saying earlier, and so they enslave men who are better by nature of the same and the same such a state and what what is the same the same the same the same the same the some the same and something and so the same stay is what is the same the same the same property and what we say the same and the same the same the same the same the some of the same the same and the same has some other the same the same the same complete and the man the same some state of the 
348/348 [==============================] - 22s 52ms/step - loss: 2.1827 - categorical_crossentropy: 2.1827 - accuracy: 0.3768
Epoch 2/50
346/348 [============================>.] - ETA: 0s -

/tmp/ipykernel_1185061/1209408919.py:4: RuntimeWarning: divide by zero encountered in log
  preds = np.exp(np.log(preds) / temperature)  # softmax


the same as the other things that are all the same as the same as the same as the same and the same as a good people who are all the other than the others and the proper and the common laws and the other things that are all the same as the same thing as the same as the same as a soul that is to be t
348/348 [==============================] - 17s 50ms/step - loss: 1.1396 - categorical_crossentropy: 1.1396 - accuracy: 0.6476
Epoch 18/50
348/348 [==============================] - ETA: 0s - loss: 1.1370 - categorical_crossentropy: 1.1370 - accuracy: 0.6485
----- Generating text after Epoch: 17
----- diversity: 0.25
----- Generating with seed: "earning easily, wallowing in it like a pig? And with regard to moderation, courage, high mindedness,"
earning easily, wallowing in it like a pig? And with regard to moderation, courage, high mindedness, and the state of the soul which is also a state and are not the same as the body and the other things that are and what is the same as we say that th

ourt and to teach other people both how to deliver and how to compose the sort of speeches suitable to the soul that is in control of the Greeks and the other things that are as the same things are not is the same as the constitution of the soul to the same point and the other things that are considered the same as the same thing, and in the same way as it is to be a friend to the same point and the other things that are the same as the soul that is the same as the other things that are to be a 
348/348 [==============================] - 18s 51ms/step - loss: 1.1200 - categorical_crossentropy: 1.1200 - accuracy: 0.6530
Epoch 27/50
347/348 [============================>.] - ETA: 0s - loss: 1.1182 - categorical_crossentropy: 1.1182 - accuracy: 0.6534
----- Generating text after Epoch: 26
----- diversity: 0.25
----- Generating with seed: "what he doesn't know. Wasn't that what we just said? And isn't that what you Pharsalians were doing "
what he doesn't know. Wasn't that what we just sai